<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221024_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전에 사용한 항공 데이터를 가지고 실습해 볼 것임

실습하려면 항공 데이터 table이 필요함

csv를 연다

create 문을 사용해서 테이블 생성(airline_delay) // 숫자는 int, 문자열은 varchar 아닌 String으로

연도별로 파티션을 나누어 저장

create table airline_delay(
    YEAR INT, MONTH INT, DAY_OF_MONTH INT, DAY_OF_WEEK INT, FL_DATE STRING, UNIQUE_CARRIER STRING, TAIL_NUM STRING, FL_NUM INT, ORIGIN_AIRPORT_ID INT, ORIGIN STRING, ORIGIN_STATE_ABR STRING, 
    DEST_AIRPORT_ID INT, DEST STRING, DEST_STATE_ABR STRING, CRS_DEP_TIME STRING, DEP_TIME STRING, DEP_DELAY INT, DEP_DELAY_NEW INT, DEP_DEL15 INT, DEP_DELAY_GROUP INT, TAXI_OUT INT,
    WHEELS_OFF STRING, WHEELS_ON STRING, TAXI_IN INT, CRS_ARR_TIME STRING, ARR_TIME STRING, ARR_DELAY INT, ARR_DELAY_NEW INT, ARR_DEL15 INT, ARR_DELAY_GROUP INT, CANCELLED INT,
    CANCELLATION_CODE STRING, DIVERTED INT, CRS_ELAPSED_TIME INT, ACTUAL_ELAPSED_TIME INT, AIR_TIME INT, FLIGHTS INT, DISTANCE INT, DISTANCE_GROUP INT, CARRIER_DELAY STRING,
    WEATHER_DELAY STRING, NAS_DELAY STRING, SECURITY_DELAY STRING, LATE_AIRCRAFT_DELAY STRING)
partitioned by (delay_year int)
row format delimited
fields terminated by ','
lines terminated by '\n'
stored as textfile;
  File "<ipython-input-1-bdb74ae9ba41>", line 1
    create table airline_delay(YEAR INT, MONTH INT, DAY_OF_MONTH INT, DAY_OF_WEEK INT, FL_DATE STRING, UNIQUE_CARRIER STRING, TAIL_NUM STRING, FL_NUM INT, ORIGIN_AIRPORT_ID INT, ORIGIN STRING, ORIGIN_STATE_ABR STRING,
               ^
SyntaxError: invalid syntax
HIVE 에 table을 만들었으니 하둡에 HIVE 공간이 생김
HDFS에서 data를 HIVE 경로로 move시킬 것임

HIVE 경로
hdfs dfs -ls /user/hive/warehouse/airline_delay
원래 데이터 경로
hdfs dfs -ls air-input
원래 데이터를 HIVE에게 줘보자 (총 12개)
load data inpath 'air-input/airOT199101.csv' into table airline_delay partition (delay_year=1991);
load data inpath 'air-input/airOT199103.csv' into table airline_delay partition (delay_year=1991);
load data inpath 'air-input/airOT199106.csv' into table airline_delay partition (delay_year=1991);
load data inpath 'air-input/airOT199107.csv' into table airline_delay partition (delay_year=1991);
load data inpath 'air-input/airOT199902.csv' into table airline_delay partition (delay_year=1999);
load data inpath 'air-input/airOT199903.csv' into table airline_delay partition (delay_year=1999);
load data inpath 'air-input/airOT199910.csv' into table airline_delay partition (delay_year=1999);
load data inpath 'air-input/airOT199911.csv' into table airline_delay partition (delay_year=1999);
load data inpath 'air-input/airOT201202.csv' into table airline_delay partition (delay_year=2012);
load data inpath 'air-input/airOT201205.csv' into table airline_delay partition (delay_year=2012);
load data inpath 'air-input/airOT201209.csv' into table airline_delay partition (delay_year=2012);
load data inpath 'air-input/airOT201211.csv' into table airline_delay partition (delay_year=2012);
만약에 파티션을 잘못 나누었을 경우 드랍하면 됨
ALTER TABLE airline_delay DROP PARTITION(delay_year='1991');
기존 하둡에서 해당 data들이 move한것을 확인할 수 있음
hdfs dfs -ls air-input
HIVE에서 select 문으로 색인 가능
select year, month, day_of_month, fl_date, unique_carrier, origin, dep_time, dep_delay 
from airline_delay 
where delay_year='1991' 
limit 20;
기존의 map reduce 작성시 보다 간결하고 빠르게 SQL문으로 색인 가능
select year, month, count(*) as delay_count 
from airline_delay 
where dep_delay > 0 
group by year, month 
order by year, month;